<img width="8%" alt="Growth" src="https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/growth_marketing.png" style="border-radius: 15%">

# Growth plugin

**Tags:** #naaschatplugin #naas #naas_driver #chat #plugin #ai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Description:** This notebook creates a Naas Chat plugin with commands.

## Input

### Import libraries

In [ ]:
import naas
from naas_drivers import naas_chat_plugin, gsheet
import naas_data_product
from IPython.display import Markdown
from datetime import date, timedelta
import os
import pandas as pd

### Setup variables
**Inputs**
- `entity_index`: Entity index.
- `entity_dir`: Entity directory.
- `input_dir`: Input directory to retrieve file from.
- `input_file`: Input file.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `name`: Plugin name.
- `model`: Plugin chat model.
- `temperature`: Plugin chat temperature.
- `description`: Plugin description.
- `avatar`: Plugin avatar image.
- `model_dir`: Models directory.

**Outputs**
- `output_dir`: Output directory where output files are stored. 
- `entity_name`: Entity name.

In [ ]:
# Inputs
entity_index = "0"
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "entity_dir")
input_dir = os.path.join(entity_dir, "growth-engine", date.today().isoformat())
input_file = "growth"
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "abi_spreadsheet")
sheet_name = "GROWTH"
name = "Growth Assistant"
model = "gpt-4-1106-preview"
temperature = 0.5
description = "Analyze content engagement, identifying potential leads through social media engagement, scoring interactions, and enriching profiles for targeted sales outreach."
avatar = "https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/growth_marketing.png"
model_dir = os.path.join(naas_data_product.ROOT_PATH, "models", "growth-engine")
api_key = naas.secret.get('NAAS_API_TOKEN')

# Outputs
output_dir = os.path.join(entity_dir, "plugins")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "entity_name")

## Model

### Get assets

In [ ]:
image_url = get_image_asset(input_dir, "growth_trend.png")
print("- Image URL:", image_url)

image_html = get_image_asset(input_dir, "growth_trend.html")
print("- Image HTML:", image_html)

### Get data to be sent to prompt

#### Get content published

In [ ]:
# Get content published
df_content = pd.DataFrame()
content_dir = input_dir.replace("growth-engine", "content-engine")
data_content = pload(content_dir, "content")
if data_content is None:
    data_content = gsheet.connect(spreadsheet_url).get(sheet_name="POSTS")
if isinstance(data_content, pd.DataFrame):
    df_content = data_content.copy()
    
if len(df_content) > 0:
    df_content = df_content[(df_content["ENTITY"] == entity_name) & (df_content["SCENARIO"].isin([TW, LW]))]
    df_content = df_content[["TITLE", "URL", "DATE", "VIEWS", "ENGAGEMENTS"]]

# Transform df to string
post_data = df_content.to_string()

# Count tokens
post_tokens = naas_chat_plugin.num_tokens_from_string(post_data)
print(f"- Post tokens: {post_tokens}")

#### Get leads generated

In [ ]:
# Init
prompt_data = []
total_leads = 0
new_leads = 0

# Load data from pickle
data = pload(input_dir, input_file)    
if data is None:
    data = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
if isinstance(data, pd.DataFrame):
    df = data.copy()

# Filter on this week and last week
if len(df) > 0:
    df = df[(df["ENTITY"].str.contains(entity_name)) & (df["SCENARIO"].isin([TW, LW]))]

    # Get data
    total_leads = len(df)
    new_leads = len(df[(df["SCENARIO"] == TW)])

    # Filter to not exceed context windowns
    columns = [
        'FULLNAME',
        'OCCUPATION',
        'INTERACTION_SCORE',
        'LAST_INTERACTION_DATE',
        'PROFILE_URL',
    ]
    prompt_data = df[columns]
    if len(prompt_data) > 0:
        prompt_data = prompt_data[:100]
            
print(f"- Leads: {total_leads}")
print(f"- New Leads: {new_leads}")
print(f"- Growth data: {len(prompt_data)}")

### Create system prompt

In [ ]:
system_prompt = f"""
Act as a Growth assistant who has access to a list of interactions from content that enable the user to get marketing qualified contacts.
Your role is to manage and optimize the list of people who interacted on the content, ensuring to extract only the most qualified contacts to feed the sales representative.
The last 2 weeks '{TW}' & '{LW}' the '{entity_name}' generated {total_leads} new leads ({new_leads} on {TW}) by the following posts published: {post_data}. 
Here is the list of leads with the highest interaction score: {df.to_string()}.
Please remember the user that you don't have access to all the list due to the limit of the context window.
The first message should be about presenting yourself and analyze briefly '{entity_name}' new interactions with a maximum of 3 bullet points.
Then, you will display the image inside the markdown of the chat about the contacts reached over the weeks: ![Contacts Reached]({image_url}).
You must ALWAYS show the image in the first message.
Be casual, but professional. Wait for the first answer from the user, and then start with the first high-level analysis. 
"""

## Output

### Create Naas Chat plugin & Push to Workspace

In [ ]:
# Create commands
print("--> Creating commands")
commands = create_command(model_dir)
print()

# Create Naas Chat plugin
print("--> Creating Naas Chat plugin")
plugin_file_path, plugin_url, plugin = create_plugin(
    name=name,
    prompt=system_prompt,
    model=model,
    temperature=temperature,
    output_dir=output_dir,
    description=description,
    avatar=avatar,
    commands=commands,
    entity_name=entity_name,
)
print()

# Push plugin to Personal workspace
print("--> Push plugin to Personal workspace")
push_plugin_to_workspace(api_key, plugin)
print()

# Create new chat
print("--> Create new chat")
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")